In [14]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import tempfile
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from datetime import datetime


In [15]:
from station.data_submission import DataSubmission
from station.station import StationData


from station.dat_to_nc_all_var import DatToNcAllVar


In [16]:
# Initialize the converter
converter = DatToNcAllVar(
    name="Vienna_AllVar",
    directory="measurements/Vienna",
    target_directory="station_data_as_nc",
    hourly=True,
    keep_original=True
)


In [17]:
converter.execute()

Extracting Vienna_AllVar...


100%|██████████| 3714/3714 [00:38<00:00, 97.19it/s] 


Saving to station_data_as_nc/vienna_allvar.nc


/Users/ferdinand/Desktop/MasterThesis/digital-twin-for-weather-station/station/dat_to_nc_all_var.py:309: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  self.dataframe[col] = pd.to_numeric(self.dataframe[col], errors='ignore')


                     bmp180_temp  bmp180_pres  bmp180_slp  bmp180_alt  \
datetime                                                                
2017-04-28 00:00:00      -999.99      -999.99     -999.99     -999.99   
2017-04-28 00:01:00      -999.99      -999.99     -999.99     -999.99   
2017-04-28 00:02:00      -999.99      -999.99     -999.99     -999.99   
2017-04-28 00:03:00      -999.99      -999.99     -999.99     -999.99   
2017-04-28 00:04:00      -999.99      -999.99     -999.99     -999.99   

                     bmp280_temp  bmp280_pres  bmp280_slp  bmp280_alt  \
datetime                                                                
2017-04-28 00:00:00         6.48       992.04     1011.47       159.0   
2017-04-28 00:01:00         6.41       992.04     1011.47       159.0   
2017-04-28 00:02:00         6.40       992.09     1011.52       159.0   
2017-04-28 00:03:00         6.42       991.97     1011.40       159.0   
2017-04-28 00:04:00         6.36       991.98     